In [1]:
import weaviate
from weaviate.config import AdditionalConfig
from weaviate.classes.config import Configure

import json
import pandas as pd
import numpy as np
import os
import dotenv

dotenv.load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')
AZURE_OPENAI_RESOURCE_NAME = os.getenv('AZURE_OPENAI_RESOURCE_NAME')


client = weaviate.connect_to_local(
    port=8083,
    grpc_port=50051,
    headers = {
        "X-Azure-Api-Key": AZURE_OPENAI_API_KEY,
    },
)

print(client.is_ready())


True


In [2]:
data = pd.read_parquet('/home/isma/repos/book/data-pipelines-with-airflow-2nd-ed/chapter13_genai/recipe_book/notebooks/splitted_with_vectors.parquet')
data

,filename,document_sha,chunk_sha,chunk,vector
0,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,c001f008-b5df-5ec0-8905-17ae645171e6,Empanadas argentinas,"[-0.006259406451135874, -0.005043736658990383,..."
1,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,8830dfd7-1331-5221-8a08-5f8cc7c46c49,Ingredientes Medidas 4 Porciones Seleccionar t...,"[-0.0013423336204141378, 0.008130612783133984,..."
2,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,9c95627f-1e0c-5a5d-8bdd-eb4e9cbb7da0,"1 taza de cebolla, finamente picada, para el r...","[0.0017024519620463252, -0.0022656326182186604..."
3,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,b88038c3-6087-52c9-a2df-dffef9bff309,"1 taza de huevo duro, finamente picado, para e...","[-0.004608715884387493, -0.007404274772852659,..."
4,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,1f433cc2-34d7-511d-addd-f8660e850f36,"suficiente de harina, para extender la masa su...","[-0.014491851441562176, -0.002449954394251108,..."
5,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,c60774cf-e659-5237-a63e-3bd0aea3ccfb,Para el relleno de carne: calienta el aceite d...,"[-0.005385186523199081, -0.007785658817738295,..."
6,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,b839d3a7-4daf-5460-975c-cb76ee3dc321,Para el relleno de elote: derrite la mantequil...,"[-0.0181081872433424, -0.008491011336445808, -..."
7,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,e554b863-d17a-588f-9210-b05955c6f34b,Barniza las empanadas argentinas con el huevo ...,"[0.0062514375895261765, -0.0070929154753685, -..."
8,pavo-a-la-hawaiana.txt,3303775d-98cc-5c62-adef-be3dce52c5a4,54af8361-016d-565e-9803-dccbc11fb95d,Pavo a la hawaiana Ingredientes 1 taza de sals...,"[-0.013167427852749825, -0.0001442846114514395..."
9,pavo-a-la-hawaiana.txt,3303775d-98cc-5c62-adef-be3dce52c5a4,367213bc-c570-5935-be25-ed7993c225b8,"Preparación En un bowl, combina la salsa de so...","[-0.012540347874164581, -0.019292842596769333,..."


In [18]:
COLLECTION_NAME = "recipes"

existing_collections = [item.lower() for item in list(client.collections.list_all().keys())]

if COLLECTION_NAME.lower() in existing_collections:

    client.collections.delete(COLLECTION_NAME)


collection = client.collections.create(
    COLLECTION_NAME,
    vectorizer_config=[
        Configure.NamedVectors.text2vec_azure_openai(
            name="vector",
            source_properties=["chunk"],
            base_url= AZURE_OPENAI_ENDPOINT,
            resource_name=AZURE_OPENAI_RESOURCE_NAME,
            deployment_id="text-embedding-3-large",
        )    
    ],





)

collection.config.get().to_dict()


{'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
  'cleanupIntervalSeconds': 60,
  'indexNullState': False,
  'indexPropertyLength': False,
  'indexTimestamps': False,
  'stopwords': {'preset': 'en'}},
 'multiTenancyConfig': {'enabled': False,
  'autoTenantCreation': False,
  'autoTenantActivation': False},
 'properties': [],
 'replicationConfig': {'factor': 1,
  'asyncEnabled': False,
  'deletionStrategy': 'DeleteOnConflict'},
 'shardingConfig': {'virtualPerPhysical': 128,
  'desiredCount': 1,
  'actualCount': 1,
  'desiredVirtualCount': 128,
  'actualVirtualCount': 128,
  'key': '_id',
  'strategy': 'hash',
  'function': 'murmur3'},
 'vectorConfig': {'vector': {'vectorizer': {'text2vec-openai': {'baseURL': 'https://slackgpt-openai-fr.openai.azure.com/',
     'deploymentId': 'text-embedding-3-large',
     'model': 'ada',
     'resourceName': 'slackgpt-openai-fr',
     'vectorizeClassName': True,
     'properties': ['chunk']}},
   'vectorIndexConfig': {'cleanupIntervalSeconds':

In [6]:
COLLECTION_NAME = "recipes"
collection = client.collections.get(COLLECTION_NAME)


source_objects = json.loads( 
    data
    .drop(columns=["document_sha"])
    .to_json(orient="records")
) 


with collection.batch.dynamic() as batch:
    for src_obj in source_objects:
        properties_obj = {
            "filename": src_obj["filename"],
            "chunk": src_obj["chunk"],
        }

        # The model provider integration will automatically vectorize the object
        batch.add_object(
            properties=properties_obj,
            uuid=src_obj["chunk_sha"],
        )


In [8]:

response = collection.query.near_text(
    query="Como hacer empanadas",  # The model provider integration will automatically vectorize the query
    limit=2
)

for obj in response.objects:
    print(obj.properties["chunk"])

Para el relleno de carne: calienta el aceite de oliva en un sartén y agrega la cebolla, el ajo y el pimiento rojo y cocina por 5 minutos. Incorpora la carne molida, la sal, la pimienta, el orégano y el comino y cocina por 10 minutos o hasta que la carne esté muy bien cocida. Retira del fuego y mezcla con el huevo y las aceitunas. Deja enfriar.
1 taza de huevo duro, finamente picado, para el relleno de carne 1/4 tazas de aceituna, finamente picada, para el relleno de carne 2 cucharadas de mantequilla, en cubos, para el relleno de elote 1/2 tazas de cebolla, finamente picada, para el relleno de elote 2 tazas de elote amarillo, desgranado, para el relleno de elote 2 tazas de queso manchego, rallado, para el relleno de elote suficiente de harina, para extender la masa suficiente de huevo, para barnizar


In [21]:
for item in collection.iterator():
    print(item.uuid, item.properties)

1f433cc2-34d7-511d-addd-f8660e850f36 {'filename': 'empanadas-argentinas.txt', 'chunk': 'suficiente de harina, para extender la masa suficiente de huevo, para barnizar chimichurri al gusto, para acompañar Preparación En un bowl, mezcla la harina, la sal, polvo para hornear, la mantequilla y el vinagre hasta integrar muy bien la mantequilla, después vierte el agua poco a poco y mezcla hasta formar una masa. Cuando la masa tenga una consistencia homogénea, tapa con un trapo y deja reposar por 20 minutos.'}
367213bc-c570-5935-be25-ed7993c225b8 {'filename': 'pavo-a-la-hawaiana.txt', 'chunk': 'Preparación En un bowl, combina la salsa de soya, la cátsup, el azúcar mascabado, el almíbar de la piña, el caldo de pollo, el jengibre y el ajo. Marina las Fajitas de Pavo Dos Familias con la mezcla anterior en refrigeración durante 20 minutos. Calienta un poco de aceite en una sartén, añade la piña y las Fajitas de Pavo Dos Familias marinadas. Cocina por 15 minutos o hasta que estén bien cocidas y la

In [22]:
collection.data.delete_by_id(
    '9c95627f-1e0c-5a5d-8bdd-eb4e9cbb7da0'
)

True

In [13]:
for item in collection.iterator():
    print(item.uuid, item.properties)

WeaviateQueryError: Query call with protocol GRPC search failed with message <AioRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "could not find class Recipes in schema"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"could not find class Recipes in schema", grpc_status:2, created_time:"2024-10-25T16:31:20.36369135+02:00"}"
>.

In [12]:
client.collections.delete(COLLECTION_NAME)